In [8]:
import pandas as pd

In [103]:
df_doqa_train = pd.read_json('/content/doqa-cooking-train-v2.1.json', orient='split')
df_doqa_train['split'] = ['train'] * len(df_doqa_train)
df_doqa_dev = pd.read_json('/content/doqa-cooking-dev-v2.1.json', orient='split')
df_doqa_dev['split'] = ['validation'] * len(df_doqa_dev)
df_doqa_test = pd.read_json('/content/doqa-cooking-test-v2.1.json', orient='split')
df_doqa_test['split'] = ['test'] * len(df_doqa_test)

df_doqa_raw = pd.concat([df_doqa_train, df_doqa_dev, df_doqa_test]).reset_index()

df_doqa_raw['paragraphs'] = df_doqa_raw['paragraphs'].apply(lambda x: x[0])


In [112]:
df_doqa_raw

,index,title,background,paragraphs,split
0,0,Tips for grilling duck legs?,I recently attempted to grill duck legs on my ...,{'context': 'I think grilling is probably a ba...,train
1,1,Tips for grilling duck legs?,I recently attempted to grill duck legs on my ...,{'context': 'I think grilling is probably a ba...,train
2,2,Tips for grilling duck legs?,I recently attempted to grill duck legs on my ...,{'context': 'I think grilling is probably a ba...,train
3,3,Meaning of do not thaw for frozen food,Today I bought frozen food for the first time ...,{'context': 'Your question is slightly confusi...,train
4,4,Meaning of do not thaw for frozen food,Today I bought frozen food for the first time ...,{'context': 'Your question is slightly confusi...,train
...,...,...,...,...,...
1632,395,First onion or first minced meat?,"When making e.g. spaghetti sauce, do I:first s...",{'context': 'In order to develop browning for ...,test
1633,396,Metal dust/shavings in food from knife wear?,"I've been wondering about this for a while, si...",{'context': 'At a microscopic level metal is m...,test
1634,397,How to make corn nuts from fresh corn on the cob,"If I have some fresh, raw corn on the cob, how...",{'context': 'You fry them in a saucepan with s...,test
1635,398,Emergency Coffee via Chocolate Covered Coffee ...,"Scenario:I am out of coffee, both whole beans ...",{'context': 'It probably would not damage anyt...,test


In [114]:
contexts = []
questions = []
answers = []
answer_starts = []
history = []
split = []

for i, row in df_doqa_raw.iterrows():
    qas = row['paragraphs']['qas']
    for j, qa in enumerate(qas):
        split.append(row['split'])
        contexts.append(row['paragraphs']['context'])
        questions.append(qa['question'])
        answers.append(qa['answers'][0]['text'])
        answer_starts.append(qa['answers'][0]['answer_start'])

df_doqa = pd.DataFrame(data={'split': split, 'context': contexts, 'question': questions, 'answer': answers, 'answer_start': answer_starts})
df_doqa['answer'] = df_doqa['answer'].apply(lambda x: x.replace('CANNOTANSWER', 'unanswerable'))
df_doqa['turn'] = df_doqa.apply(lambda row: f"student: {row['question']} | teacher: {row['answer']} ", axis=1)
df_doqa['id'] = list(range(len(df_doqa)))
df_doqa['prev_id'] = df_doqa['id'] - 1
df_doqa['prev_prev_id'] = df_doqa['prev_id'] - 1

df_temp = df_doqa[['id', 'context', 'turn']].rename(columns={'turn': 'prev_turn'})
df_doqa = df_doqa.merge(df_temp, right_on=['id', 'context'], left_on=['prev_id', 'context'], how='left')
df_temp = df_temp.rename(columns={'prev_turn': 'prev_prev_turn'})
df_doqa = df_doqa.merge(df_temp, right_on=['id', 'context'], left_on=['prev_prev_id', 'context'], how='left')

df_doqa['prev_turn'] = df_doqa['prev_turn'].fillna('')
df_doqa['prev_prev_turn'] = df_doqa['prev_prev_turn'].fillna('')
df_doqa['history'] = df_doqa['prev_turn'] + df_doqa['prev_prev_turn']

df_doqa = df_doqa[['split', 'context', 'question', 'answer', 'answer_start', 'history']]
df_doqa['input'] = df_doqa.apply(lambda row: f"{row['question']}SPLITContext: {row['context']}\nHistory: {row['history']}", axis=1)
df_doqa['outpupt'] = df_doqa['answer']

In [115]:
df_doqa.to_excel('/content/doqa_wot_format.xlsx')